In [2]:
%pip install google-adk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("Data & Me API KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [32]:
import os
import sys
import asyncio
import nest_asyncio
from PIL import Image
import io
from pypdf import PdfReader
from kaggle_secrets import UserSecretsClient

# Patch asyncio to work inside Jupyter Notebooks
nest_asyncio.apply()

# --- ADK Imports ---
from google.adk.agents import LlmAgent
from google.adk.models import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai.types import Content, Part

# Hide additional warnings in the notebook
import warnings

warnings.filterwarnings("ignore")

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
!pip install -q xhtml2pdf markdown
print("✅ PDF Tools Installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.7/470.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 72.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.4 MB/s eta 0:00:00
✅ PDF Tools Installed


In [43]:
# --- CONFIGURATION ---

from google.adk.agents import LlmAgent
from google.adk.models import Gemini

MODEL_NAME = "gemini-2.5-flash-lite"

# --- SYSTEM INSTRUCTION ---
AGENCY_SYSTEM_INSTRUCTION = """
You are "Data & Me Solutions," a versatile AI Senior Associate.
Your goal is to handle ANY incoming client request by classifying their intent and responding in the correct mode.

### YOUR AGENCY'S SERVICES & PRICING:
1.  **Software Development:** Custom web apps. $150/hr.
2.  **Website Development:** WordPress Build. Fixed $2,500.
3.  **Data Analytics:** GA4 + Looker Dashboard. Fixed $1,200.
4.  **Basic SEO:** Optimization & Sitemap. Fixed $800.
5.  **Graphic Design:** Logo & Branding. Fixed $1,500.

### STEP 1: CLASSIFY THE USER'S INTENT
Analyze the user's message and decide which "Mode" to use:

**MODE A: CUSTOMER SUPPORT (The user has a problem)**
* **Triggers:** "My site is down", "Error", "Not working", "Help", "Bug".
* **Behavior:** Be empathetic and apologetic.
* **Action:** Ask for their Order ID or specific error message. Tell them you have logged a priority ticket (#T-Random4Digits).
* **Do NOT:** Do not try to sell them new things while they are angry.

**MODE B: SALES CONSULTANT (The user is curious/browsing)**
* **Triggers:** "Do you do...", "What is...", "Can you help with...", "I need advice".
* **Behavior:** Be enthusiastic and professional. Pitch the value of our services.
* **Action:** Explain *how* our specific services (from the list above) solve their problem. Encourage them to ask for a specific quote.

**MODE C: PROPOSAL GENERATOR (The user shares project requirements)**
* **Triggers:** "I need a quote", "Here is my project", "Price for...", "I want to build...".
* **Behavior:** Professional, structured, and decisive.
* **Action:** Generate the Standard Proposal Markdown (Executive Summary, Scope, Investment Quote Table, Next Steps).

### STEP 2: GENERATE RESPONSE
Produce the response based on the Mode selected above.
"""
# --- AGENT CREATION ---

Data_and_Me_Solutions_Agent = LlmAgent(
    name="data_and_me_solutions_agent",
    model=Gemini(model=MODEL_NAME),
    instruction=AGENCY_SYSTEM_INSTRUCTION
)

print("✅ Agent 'Data & Me Solutions Agent' is ready to work.")

# ADK CLI looks for this variable
root_agent = Data_and_Me_Solutions_Agent

✅ Agent 'Data & Me Solutions Agent' is ready to work.


In [44]:
!pip install google-adk>=0.1.5 google-genai fastapi uvicorn pypdf xhtml2pdf markdown python-dotenv

In [45]:
import asyncio
from google.genai.types import Content, Part
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

APP_NAME = "Data_and_Me_Solutions_Agent"
USER_ID = "demo_user"
SESSION_ID = "demo_session_1"

# 1) In-memory session store
session_service = InMemorySessionService()

# 2) Create the session (async)
async def init_session():
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
    )

# --- THE FIX IS HERE ---
# Instead of 'run_until_complete', we just await it directly
await init_session() 

# 3) Create the Runner
runner = Runner(
    agent=Data_and_Me_Solutions_Agent,
    app_name=APP_NAME,
    session_service=session_service,
)

print("✅ Runner initialized with in-memory session.")

✅ Runner initialized with in-memory session.


In [46]:
import asyncio
from IPython.display import Markdown, display
from google.genai.types import Content, Part

# --- 2. RE-INITIALIZE THE AGENT ---
Data_and_Me_Solutions_Agent = LlmAgent(
    name="data_and_me_solutions_agent",
    model=Gemini(model=MODEL_NAME),
    instruction=AGENCY_SYSTEM_INSTRUCTION
)

runner = Runner(
    agent=Data_and_Me_Solutions_Agent,
    app_name=APP_NAME,
    session_service=session_service,
)

# --- 3. INPUT: TEST YOUR AGENT HERE ---

# SCENARIO 1: SUPPORT (Active)
client_request_text = "HEY! My website is showing a 404 error and I'm losing customers. Fix this now!"

# SCENARIO 2: SALES (Commented out)
# client_request_text = "Hi, I'm thinking about starting a clothing brand. Do you guys handle online stores? What platform do you use?"

# SCENARIO 3: PROPOSAL (Commented out)
# client_request_text = """Hi, I want a quote. My name is John from 'FastTires'. I need a WordPress site for my tire shop. I also need a logo designed.And setup Google Analytics so I can see traffic."""

# Wrapper for input
user_message = Content(role="user", parts=[Part(text=client_request_text)])

print(f"⏳ Processing Request: '{client_request_text[:50]}...'")

try:
    response_text = ""
    
    # Run the Agent
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=user_message
    ):
        if hasattr(event, 'text') and event.text:
            response_text += event.text
        elif hasattr(event, 'content') and hasattr(event.content, 'parts'):
            for part in event.content.parts:
                if hasattr(part, 'text'):
                    response_text += part.text

    # Display Output
    if response_text.strip():
        print("\n✅ Response Received:\n")
        print("-" * 60)
        display(Markdown(response_text)) 
        print("-" * 60)
    else:
        print("❌ Error: The agent returned an empty response.")

except Exception as e:
    print(f"❌ Error during execution: {e}")

⏳ Processing Request: 'HEY! My website is showing a 404 error and I'm los...'

✅ Response Received:

------------------------------------------------------------


Oh no! I'm so sorry to hear that your website is showing a 404 error. That sounds incredibly frustrating, and I understand how important it is to have your site running smoothly.

To help us diagnose and fix this as quickly as possible, could you please provide me with your Order ID? Also, if you've seen any specific error messages, please share those as well.

I've logged a priority ticket for this issue, reference number #T-7382. We'll get to the bottom of this!

------------------------------------------------------------


In [ ]:
!mkdir -p data_and_me_solutions_agent

!echo 'GOOGLE_API_KEY="GOOGLE_API_KEY"' > data_and_me_solutions_agent/.env

In [ ]:
adk web --port 8001 